# Laboratotio 6

- Nelson García
- Ricardo Chuy
- Christian Echeverría

# Task 1 - Teoría

### 1. En un juego de suma cero para dos jugadores, ¿cómo funciona el algoritmo minimax para determinar la
estrategia óptima para cada jugador? ¿Puede explicarnos el concepto de "valor minimax" y su importancia
en este contexto?

Lo que ocurre es que en un juego de suma cero, la suma de la utilidad de ambos jugadores da 0. Digamos si gana el jugador se le da 1 y al oponente -1 respectivamente. El minimax es una estrategia que se utiliza para minimizar la ganancia del contricante para poder tomar la mejor decisión en un turno. Con minimax lo que se hace es explorar el árbol de las posibles jugadas dado el estado actual del juego, intentando minimizar las ganancias del contricante. Hay nodos maximizadores y otros minimizadores, a veces se elije la jugada que tiene el valor minimo y posteriormente en el siguiente nivel, el máximo, de allí biene el nombre del algoritmo "minimax". Ahora con respecto al valor Minimax este representa el mejor resultado que puede obtener un jugador. Este se obtiene por medio de la exploración recursiva del árbol para obtener el mejor resultado. Este es muy importante ya que permite que los jugadores puedan realizar las acciones y decisiones en un punto del juego. Maximizando su ganancia y minimizando la del rival.

### 2. Compare y contraste el algoritmo minimax con la poda alfa-beta. ¿Cómo mejora la poda alfa-beta la
eficiencia del algoritmo minimax, particularmente en árboles de caza grandes? Proporcione un ejemplo
para ilustrar la diferencia en la complejidad computacional entre la poda minimax y alfa-beta.

El algoritmo minimax es bastante bueno, pero tiene el problema que cuando crece el entorno y la complejidad del juego también lo hace el arbol. Por ejemplo, el árbol de jugadas que se hace para un estado específico de un juego de totito es mucho más pequeño que el de un juego de ajedres. Es más, el árbol de ajedrés tiene tantas posibles jugadas en un estado del tablero, que sería prácticamente imposible utilizar minimax para recorrerlo completamente y obtener a mejor jugada.  Aquí es donde entra el algoritmo alfa beta pruning, que lo que hace principalmente es simplificar significativamnete el análisis del árbol. Si al momento de analizar una parte del árbol se encuentra un nodo que ya no se puede mejorar más se "podan" (no se exploran) las ramas restantes ya que no es neceario revisarlas. La complejidad de minimax es de O(b^d), pero la complejidad de alfa beta pruning es de O(b^(d/2)), donde b es el factor de ramificación (que tantas rama salen de cada nodo) y d es la profundidad del árbol. Eso quiere decir que si tenemos un juego donde el factor de ramificación es de 10 con profundidad 4, se evaluan aproximadamente 10^4 = 10000sin poda alfa-beta, mientras que con poda sería 10^(4/2) = 100. Una mejora bastante grande.

### 3. ¿Cuál es el papel de expectiminimax en juegos con incertidumbre, como aquellos que involucran nodos de azar o información oculta? ¿En qué se diferencia el expectiminimax del minimax en el manejo de resultados probabilísticos y cuáles son los desafíos clave que aborda?

El expectiminimax es una versión extendida del minimax que se usa principalmente cuando en un juego hay fáctores de incertidumbre (como eventos aleatorios) antes o después de tomar una decisión. El papel de este algoritmo es poder manejar y tomar decisiones óptimas para el jugador cuando el entorno realmente no es determinista. Es útil para tomar la mejor decisión posible cuando las decisiones y resultados dependen de probailidades al lanzar un dado, una moneda, revelear una carta oculta, entre otros.
La principal diferencia con el minimax, es que el minimax maneja resultados deterministas, pero el expectiminimax tiene en cuenta la aleatoriedad y se calculan valores esperados de las decisiones. Es por esto que el expectiminimax toma en consideración Nodos de azar que son eventos probabilísticos, no solo nodos de minimización y maximización.
Los desafíos clave realmente tienen que ver también con la complejidad que introduce al problema los eventos aleatorios. Estos eventos hacen que las situaciones no sean deterministas y se tengan que hacer estimados. Un desafío clave es poder seguir tomando decisiones óptimas incluso cuando hay incertidumbre en el problema, la complejidad computacional debido a la introducción de nuevos nodos y la función de evaluación para manejar correctamente la incertidumbre.

# Task 2 - Connect Four


Para esta Task elegimos la opcion de poder usar un ambiente generado por IA, en este caso utilice DeepSeek para la creacion del ambiente junto con algunos ajustes propios para poder visualizar el tablero. Prompt utilizado: Me puedes ayudar a crear un ambiente que simulee el juego de connect 4 para poder usar un agente que use Minimax, o sea quiero el ambiente por ahora con las reglas y recompensas

In [1]:
import numpy as np
import math
import random

Funciones de utilidad:

In [2]:
def check_win_board(board, piece):
    rows, cols = board.shape
    # Comprobar horizontal
    for r in range(rows):
        for c in range(cols - 3):
            if board[r][c] == piece and board[r][c+1] == piece and \
               board[r][c+2] == piece and board[r][c+3] == piece:
                return True
    # Comprobar vertical
    for r in range(rows - 3):
        for c in range(cols):
            if board[r][c] == piece and board[r+1][c] == piece and \
               board[r+2][c] == piece and board[r+3][c] == piece:
                return True
    # Diagonal con pendiente negativa (hacia arriba)
    for r in range(3, rows):
        for c in range(cols - 3):
            if board[r][c] == piece and board[r-1][c+1] == piece and \
               board[r-2][c+2] == piece and board[r-3][c+3] == piece:
                return True
    # Diagonal con pendiente positiva (hacia abajo)
    for r in range(rows - 3):
        for c in range(cols - 3):
            if board[r][c] == piece and board[r+1][c+1] == piece and \
               board[r+2][c+2] == piece and board[r+3][c+3] == piece:
                return True
    return False

def get_valid_locations(board):
    valid_locations = []
    cols = board.shape[1]
    for col in range(cols):
        if board[0][col] == 0:
            valid_locations.append(col)
    return valid_locations

def get_next_open_row(board, col):
    rows = board.shape[0]
    for r in range(rows - 1, -1, -1):
        if board[r][col] == 0:
            return r
    return None

Tenemos nuestro entorno de conecta 4:

In [3]:
class Connect4Env:
    def __init__(self):
        self.rows = 6
        self.cols = 7
        self.board = np.zeros((self.rows, self.cols), dtype=int)
        self.current_player = 1  # Jugador 1 comienza
        self.game_over = False
        self.winner = None

    def reset(self):
        self.board = np.zeros((self.rows, self.cols), dtype=int)
        self.current_player = 1
        self.game_over = False
        self.winner = None
        return self.board

    def is_valid_move(self, col):
        return self.board[0][col] == 0

    def make_move(self, col):
        if not self.is_valid_move(col):
            return False

        # Se deposita la ficha en la fila más baja disponible
        for row in range(self.rows - 1, -1, -1):
            if self.board[row][col] == 0:
                self.board[row][col] = self.current_player
                break

        # Se guarda el jugador que realizó el movimiento
        player_moved = self.current_player

        # Verifica si el movimiento produjo victoria o empate
        if check_win_board(self.board, player_moved):
            self.game_over = True
            self.winner = player_moved
        elif self.is_board_full():
            self.game_over = True
            self.winner = 0  # Empate

        # Cambiar turno (1 <-> 2)
        self.current_player = 3 - self.current_player
        return True

    def is_board_full(self):
        return np.all(self.board != 0)

    def render(self):
        # Imprime el tablero de forma que la base quede abajo
        #print(np.flip(self.board, 0))
        print(self.board)


Funciones de heurística:

In [4]:
def score_window(window, piece):
    score = 0
    opp_piece = 3 - piece
    if window.count(piece) == 4:
        score += 100
    elif window.count(piece) == 3 and window.count(0) == 1:
        score += 5
    elif window.count(piece) == 2 and window.count(0) == 2:
        score += 2

    if window.count(opp_piece) == 3 and window.count(0) == 1:
        score -= 4

    return score

def score_position(board, piece):
    score = 0
    rows, cols = board.shape

    # Prioridad al centro
    center_array = [int(i) for i in list(board[:, cols // 2])]
    center_count = center_array.count(piece)
    score += center_count * 3

    # Evaluar horizontales
    for r in range(rows):
        row_array = [int(i) for i in list(board[r, :])]
        for c in range(cols - 3):
            window = row_array[c:c + 4]
            score += score_window(window, piece)

    # Evaluar verticales
    for c in range(cols):
        col_array = [int(i) for i in list(board[:, c])]
        for r in range(rows - 3):
            window = col_array[r:r + 4]
            score += score_window(window, piece)

    # Evaluar diagonales (pendiente positiva)
    for r in range(rows - 3):
        for c in range(cols - 3):
            window = [board[r + i][c + i] for i in range(4)]
            score += score_window(window, piece)

    # Evaluar diagonales (pendiente negativa)
    for r in range(3, rows):
        for c in range(cols - 3):
            window = [board[r - i][c + i] for i in range(4)]
            score += score_window(window, piece)

    return score

def is_terminal_node(board):
    return check_win_board(board, 1) or check_win_board(board, 2) or len(get_valid_locations(board)) == 0


Tenemos nuestro algoritmo Minimax con opción de Alpha-Beta Pruning:

In [5]:
def minimax(board, depth, alpha, beta, maximizingPlayer, use_alpha_beta, piece):
    valid_locations = get_valid_locations(board)
    terminal = is_terminal_node(board)
    if depth == 0 or terminal:
        if terminal:
            if check_win_board(board, piece):
                return (None, 100000000000)
            elif check_win_board(board, 3 - piece):
                return (None, -100000000000)
            else:
                return (None, 0)
        else:
            return (None, score_position(board, piece))
    if maximizingPlayer:
        value = -math.inf
        best_col = random.choice(valid_locations)
        for col in valid_locations:
            row = get_next_open_row(board, col)
            b_copy = board.copy()
            b_copy[row][col] = piece
            new_score = minimax(b_copy, depth - 1, alpha, beta, False, use_alpha_beta, piece)[1]
            if new_score > value:
                value = new_score
                best_col = col
            if use_alpha_beta:
                alpha = max(alpha, value)
                if alpha >= beta:
                    break
        return best_col, value
    else:
        value = math.inf
        best_col = random.choice(valid_locations)
        for col in valid_locations:
            row = get_next_open_row(board, col)
            b_copy = board.copy()
            b_copy[row][col] = 3 - piece
            new_score = minimax(b_copy, depth - 1, alpha, beta, True, use_alpha_beta, piece)[1]
            if new_score < value:
                value = new_score
                best_col = col
            if use_alpha_beta:
                beta = min(beta, value)
                if alpha >= beta:
                    break
        return best_col, value

def get_best_move(board, depth, use_alpha_beta, piece):
    col, score = minimax(board, depth, -math.inf, math.inf, True, use_alpha_beta, piece)
    return col


Y la función main, que tiene el menú del juego:

In [17]:
def main():
    env = Connect4Env()
    env.reset()

    # Selección del modo de juego
    mode = input("Seleccione modo: 1. Jugador Humano vs IA, 2. IA vs IA: ")
    use_alpha_beta_input = input("¿Usar alpha-beta pruning? (s/N): ")
    use_alpha_beta = True if use_alpha_beta_input.lower() == 's' else False
    depth = int(input("Seleccione la profundidad de búsqueda para Minimax (por ejemplo, 4): "))

    env.render()

    while not env.game_over:
        if mode == '1':  # Humano vs IA
            if env.current_player == 1:
                # Turno del jugador humano
                valid = False
                while not valid:
                    try:
                        col = int(input("Jugador Humano, elige una columna (0-6): "))
                        if col in get_valid_locations(env.board):
                            valid = True
                        else:
                            print("Movimiento inválido, intente de nuevo.")
                    except:
                        print("Entrada no válida, intente de nuevo.")
            else:
                # Turno de la IA (jugador 2)
                print("IA está pensando...")
                col = get_best_move(env.board, depth, use_alpha_beta, 2)
                print(f"IA elige la columna {col}")
        else:  # IA vs IA
            print(f"Jugador {env.current_player} (IA) está pensando...")
            if env.current_player == 1:
                col = get_best_move(env.board, depth, False, env.current_player)
            else:
                col = get_best_move(env.board, depth, True, env.current_player)
            print(f"Jugador {env.current_player} (IA) elige la columna {col}")

        if env.make_move(col):
            print(f"Movimiento realizado en la columna {col}")
            env.render()
        else:
            print("Movimiento inválido, intente de nuevo.")

    if env.winner == 0:
        print("El juego terminó en empate.")
    else:
        print(f"El juego ha terminado. Ganador: Jugador {env.winner}")


Ahora podemos jugar contra la IA:


In [18]:
main()

Seleccione modo: 1. Jugador Humano vs IA, 2. IA vs IA: 2
¿Usar alpha-beta pruning? (s/N): s
Seleccione la profundidad de búsqueda para Minimax (por ejemplo, 4): 3
[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]]
Jugador 1 (IA) está pensando...
Jugador 1 (IA) elige la columna 3
Movimiento realizado en la columna 3
[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0]]
Jugador 2 (IA) está pensando...
Jugador 2 (IA) elige la columna 3
Movimiento realizado en la columna 3
[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 2 0 0 0]
 [0 0 0 1 0 0 0]]
Jugador 1 (IA) está pensando...
Jugador 1 (IA) elige la columna 3
Movimiento realizado en la columna 3
[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0]
 [0 0 0 2 0 0 0]
 [0 0 0 1 0 0 0]]
Jugador 2 (IA) está pensando...
Jugador 2 (IA) elige la columna 2
Movimiento realizado en la columna 2
[[0 0 0 0 0 0